In [1]:
%run init_notebookspace.py
from settings import DATA_DIR, MODEL_DIR

DATA_DIR is existant under: C:\Users\LukasGrahl\Documents\GIT\memoire1\data


In [ ]:
%matplotlib inline

from gEconpy.classes.model import gEconModel
from gEconpy.classes.transformers import IntervalTransformer
from gEconpy import compile_to_statsmodels
from gEconpy.plotting import plotting as gp


import matplotlib.pyplot as plt
import seaborn as sns
from src.plotting import plot_dfs
from src.gp_plotting import plot_sm_results

import numpy as np
import pandas as pd

import os

from config import plt_config
plt.rcParams.update(plt_config)

load model

In [ ]:
from config import mod5_params
mod = gEconModel(os.path.join(MODEL_DIR, 'gEconpy_mod5_nk.gcn'), verbose=True)
plot_vars = ['Y', 'C', 'L', 'w', 'r', 'pi']

In [ ]:
mod.blocks

In [ ]:
mod.variables

In [ ]:
# assign parameters
mod.free_param_dict.update(mod5_params)
mod.free_param_dict

Investigate priors

Solve

In [ ]:
%time mod.steady_state()

In [ ]:
# steady state values
mod.print_steady_state()

In [ ]:
mod.solve_model(solver="gensys")

In [ ]:
mod.check_bk_condition()
;

In [ ]:
# # policy matrix
# for name, policy_matrix in zip(["T", "R"], [round(mod.T, 5), round(mod.R, 5)]):
#     print(name.center(10).center(50, "="))
#     print(policy_matrix.to_string())

Model stats

In [ ]:
scov = mod.compute_stationary_covariance_matrix()
gp.plot_covariance_matrix(scov, plot_vars,
                          figsize=(5,5))
;

In [ ]:
acorr = mod.compute_autocorrelation_matrix(n_lags=20)
gp.plot_acf(acorr, plot_vars, n_cols=3, figsize=(10, 8))
;

IRF

In [ ]:
irf = mod.impulse_response_function(simulation_length=40, shock_size=1.0)
_irf = irf.copy().unstack().loc[:, "epsilon_A"].unstack()

In [ ]:
plot_dfs(_irf[plot_vars], sns.lineplot, figsize=(10, 2))

Estimation

In [ ]:
from config import recession_dict

df = pd.read_csv(os.path.join(DATA_DIR, 'prepro_data.csv'), index_col='date').drop('Unnamed: 0', axis=1)
df.index = pd.to_datetime(df.index)
df = df.asfreq('QS')

In [ ]:
df = df.rename(columns={'pi_s': 'pi'}).drop(['pi_c', 'Ix', 'Zx'], axis=1)
train = df[df['is_test'] == False].drop('is_test', axis=1).copy()
test = df[df['is_test'] == True].drop('is_test', axis=1).copy()

In [ ]:
StatsMod = compile_to_statsmodels(mod)

# parameters that we want to estimate, if not specified value will be taken from GCN file
param_start_dict = {
    'rho_A':0.85
                   }

# starting value for the shock's standard deviation
shock_start_dict = {
    'epsilon_A':0.5
                    }

# assume that the true value deviates with a σ, specified below
noise_start_dict = {
        'Y':0.01,
        'w':0.02
}

# define parameter search space e.g. 0 < α < 1, as suggested by Cobb-Douglas function
param_transforms = {
    'rho_A':IntervalTransformer(low=1e-2, high=0.99999, slope=1),
}
mod_stats = StatsMod(train[["Y", 'w', 'C']],
                 param_start_dict = param_start_dict,
                  shock_start_dict = shock_start_dict,
                  noise_start_dict = noise_start_dict,
                  param_transforms = param_transforms,
                  shock_transforms=None, 
                  noise_transforms=None, 
                  initialization='stationary')

In [ ]:
%%capture
# basinhopping as algorithm, mixes different approaches 
mod_res = mod_stats.fit(method='basinhopping')

In [ ]:
mod_res.summary()

In [ ]:
plot_sm_results(mod_res, train, var_names=plot_vars)

In [ ]:
plot_sm_results(mod_res, train, 'smoothed', var_names=plot_vars)

In [ ]:
mod_for = mod_res.get_forecast(len(test))

mu = mod_for.predicted_mean
if type(mu) == pd.Series:
    mu = pd.DataFrame(mu)

for col in mu.columns:
    fig, ax = plt.subplots(figsize=(8, 3))
    # sigma = 1.96 * np.sqrt(mod_for.var_pred_mean.loc[col, col])
    ax.plot(mu[col])
    # ax.fill_between(mu[col].index, mu[col] - sigma, mu[col] + sigma, alpha=0.15)
    ax.plot(test[col])
    ax.set_title(f'Out of sample forecast on {col}')